In [1]:
import pandas as pd
import numpy as np
from pyts.image import GramianAngularField, MarkovTransitionField
from torch.utils.data import Dataset
from typing import Callable, Tuple
import torch
from torch import Tensor, nn, optim
import torchvision
import time
import copy
import cv2
from tqdm import tqdm
from shared import InputDataset
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 64

In [2]:
df = pd.read_hdf("allfeatures_mle.hdf")
df

,profit,trade_in_1h_spot_ADAUSDT_close_price,daytime,spot_ADAUSDT_open_price_ewm_mean,spot_ADAUSDT_open_price_ewm_std,spot_ADAUSDT_open_price_ewm_cov,shift_0_spot_ADAUSDT_open_price_mean_120,shift_0_spot_ADAUSDT_open_price_skew_120,shift_0_spot_ADAUSDT_open_price_kurt_120,shift_0_spot_ADAUSDT_open_price_std_120,...,shift_0_spot_ADAUSDT_taker_buy_quote_asset_volume_kurt_60,shift_0_spot_ADAUSDT_taker_buy_quote_asset_volume_std_60,shift_240_spot_ADAUSDT_taker_buy_quote_asset_volume_mean_60,shift_240_spot_ADAUSDT_taker_buy_quote_asset_volume_skew_60,shift_240_spot_ADAUSDT_taker_buy_quote_asset_volume_kurt_60,shift_240_spot_ADAUSDT_taker_buy_quote_asset_volume_std_60,shift_480_spot_ADAUSDT_taker_buy_quote_asset_volume_mean_60,shift_480_spot_ADAUSDT_taker_buy_quote_asset_volume_skew_60,shift_480_spot_ADAUSDT_taker_buy_quote_asset_volume_kurt_60,shift_480_spot_ADAUSDT_taker_buy_quote_asset_volume_std_60
2021-01-04 11:20:59.999000+00:00,1.0,0.036722,-0.074531,-1.909687,-0.648146,-0.176138,-1.913100,-1.131116,0.623300,-0.626334,...,-0.846745,-0.630959,-0.168640,0.030476,0.021407,-0.394865,-0.661803,-0.426065,-0.297632,-0.726861
2021-01-04 11:40:59.999000+00:00,1.0,0.036938,-0.074531,-1.910256,-0.740519,-0.181543,-1.913065,-1.616506,1.056645,-0.646207,...,2.199858,-0.396540,-0.047330,-0.447464,-0.473253,-0.205394,-0.593374,-0.792412,-0.748585,-0.653821
2021-01-04 12:00:59.999000+00:00,1.0,0.021161,0.069946,-1.905108,-0.470303,-0.161243,-1.912016,-1.446041,1.229738,-0.619081,...,1.639968,-0.369021,-0.078351,-0.357117,-0.282919,-0.287197,-0.529596,-0.549676,-0.520019,-0.601402
2021-01-04 12:20:59.999000+00:00,1.0,0.013838,0.069946,-1.902825,-0.694666,-0.179059,-1.910077,-0.905540,0.416597,-0.648592,...,2.226474,-0.384426,-0.047025,-0.491028,-0.320946,-0.292472,-0.346014,-0.845703,-0.754393,-0.438681
2021-01-04 12:40:59.999000+00:00,1.0,0.011350,0.069946,-1.899724,-0.763532,-0.182642,-1.906968,0.170050,-0.562061,-0.711798,...,-0.866335,-0.610321,-0.119398,-1.151383,-0.767618,-0.438395,-0.287920,-0.932421,-0.825982,-0.426275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-29 19:33:59.999000+00:00,1.0,0.001496,1.081289,0.810568,-0.658186,-0.176803,0.820511,0.031403,-0.895658,-0.437345,...,0.763435,-0.550191,-0.037214,-0.718836,-0.686781,-0.066073,-0.639102,-0.673427,-0.630258,-0.683486
2021-10-29 19:53:59.999000+00:00,1.0,0.002985,1.081289,0.815557,-0.483507,-0.162552,0.817700,0.351614,-0.854022,-0.541512,...,-0.104784,-0.465197,-0.036275,-1.037761,-0.866607,-0.110921,-0.526061,-0.636592,-0.702661,-0.494780
2021-10-29 20:13:59.999000+00:00,0.0,-0.001986,1.225766,0.821810,-0.471555,-0.161368,0.817108,0.471610,-0.551048,-0.578451,...,-0.275976,-0.516271,-0.046592,-0.912186,-0.832715,-0.075191,-0.493173,-0.805261,-0.780498,-0.492350
2021-10-29 20:33:59.999000+00:00,1.0,0.000995,1.225766,0.820460,-0.610927,-0.173509,0.816629,0.372661,-0.450580,-0.615412,...,-0.358129,-0.512971,-0.110866,-0.538919,-0.620419,-0.148047,-0.349651,0.136156,-0.052606,-0.235194


In [3]:
X = df.iloc[:, 1:]
X.to_numpy()

array([[ 3.67216605e-02, -7.45313368e-02, -1.90968747e+00, ...,
        -4.26064644e-01, -2.97632411e-01, -7.26861132e-01],
       [ 3.69381613e-02, -7.45313368e-02, -1.91025568e+00, ...,
        -7.92412337e-01, -7.48584537e-01, -6.53821220e-01],
       [ 2.11605379e-02,  6.99461556e-02, -1.90510806e+00, ...,
        -5.49676121e-01, -5.20018733e-01, -6.01402370e-01],
       ...,
       [-1.98609732e-03,  1.22576609e+00,  8.21809641e-01, ...,
        -8.05261227e-01, -7.80498014e-01, -4.92349718e-01],
       [ 9.94530085e-04,  1.22576609e+00,  8.20459550e-01, ...,
         1.36155864e-01, -5.26062885e-02, -2.35193815e-01],
       [ 1.99302441e-03,  1.22576609e+00,  8.18192062e-01, ...,
         6.60089812e-01,  3.02636235e-01, -2.52125937e-01]])

In [4]:
Y = df.iloc[:, :1]
Y.to_numpy()

array([[1.],
       [1.],
       [1.],
       ...,
       [0.],
       [1.],
       [1.]])

In [5]:
image_size = 100
gadf = GramianAngularField(image_size=image_size, method='difference')
X_gadf = gadf.fit_transform(X)
X_gadf.shape

(21436, 100, 100)

In [6]:
validation_size = int(len(df) / 5)
dataset_train = InputDataset(df=df.iloc[:-validation_size].sample(frac=1), device=DEVICE)
dataset_valid = InputDataset(df=df.iloc[-validation_size:], device=DEVICE)


train_loader1 = torch.utils.data.DataLoader(
    dataset=dataset_train, batch_size=BATCH_SIZE, drop_last=True
)
validation_loader1 = torch.utils.data.DataLoader(
    dataset=dataset_valid, batch_size=BATCH_SIZE, drop_last=True
)

dataloaders_dict = {'train':train_loader1, 'val':validation_loader1}

In [7]:
# https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    losses = []
    class LossRecord:
        val_loss = None
        train_loss = None
        epoch = None
    
    terminate_training = False
    for epoch in range(num_epochs):
        if terminate_training:
            break
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
            
        loss_record = LossRecord()
        loss_record.epoch = epoch
        
        loss_diff = 9999999999
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            if phase == 'train':
                loss_record.train_loss = epoch_loss
                
            
            if phase == 'val':
                loss_record.val_loss = epoch_loss
                
                current_loss_diff = abs(loss_record.train_loss- loss_record.val_loss)
                if current_loss_diff > loss_diff:
                    loss_diff = current_loss_diff
                    best_model_wts = copy.deepcopy(model.state_dict())
                else:
                    terminate_training=True
            

            """
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc and epoch > 0:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
            """
            
            

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    import os 
    try:
        os.remove("best_model")
    except:
        pass
    torch.save(model.state_dict(), "best_model")
    
    return model, val_acc_history

In [8]:
# model = torchvision.models.resnet50(pretrained=True)
model = torchvision.models.resnet101(pretrained=True)
model.to(DEVICE)

"""
ct = 0
for child in model.children():
    ct += 1
    if ct < 7:
        for param in child.parameters():
            param.requires_grad = False
"""

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.00004, momentum=0.04)
#optimizer = optim.Adam(model.parameters(), lr=0.00001)

In [9]:
# this is freezing of all layers
# https://discuss.pytorch.org/t/how-the-pytorch-freeze-network-in-some-layers-only-the-rest-of-the-training/7088/3
#for param in model.parameters():
#    param.requires_grad = False

In [9]:
train_model(model, dataloaders_dict, criterion, optimizer, num_epochs=10, is_inception=False)

  0%|          | 0/267 [00:00<?, ?it/s]

Epoch 0/9
----------


/home/vaclavmatejka/anaconda3/envs/dj/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448234945/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
  0%|          | 0/66 [00:00<?, ?it/s]

train Loss: 8.9961 Acc: 0.0033


  0%|          | 0/267 [00:00<?, ?it/s]

...

 27%|██▋       | 71/267 [01:20<03:43,  1.14s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/vaclavmatejka/anaconda3/envs/dj/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-dd4588ac8c8a>", line 1, in <module>
    train_model(model, dataloaders_dict, criterion, optimizer, num_epochs=10, is_inception=False)
  File "<ipython-input-7-82de28148e1a>", line 66, in train_model
    optimizer.step()
  File "/home/vaclavmatejka/anaconda3/envs/dj/lib/python3.8/site-packages/torch/optim/optimizer.py", line 88, in wrapper
    return func(*args, **kwargs)
  File "/home/vaclavmatejka/anaconda3/envs/dj/lib/python3.8/site-packages/torch/autograd/grad_mode.py", line 28, in decorate_context
    return func(*args, **kwargs)
  File "/home/vaclavmatejka/anaconda3/envs/dj/lib/python3.8/site-packages/torch/optim/sgd.py", line 110, in step
    F.sgd(params_with_grad,
  File "/home/vaclavmatejka/anaconda3/envs/dj/lib/python3.8/site-package

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/vaclavmatejka/anaconda3/envs/dj/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-dd4588ac8c8a>", line 1, in <module>
    train_model(model, dataloaders_dict, criterion, optimizer, num_epochs=10, is_inception=False)
  File "<ipython-input-7-82de28148e1a>", line 66, in train_model
    optimizer.step()
  File "/home/vaclavmatejka/anaconda3/envs/dj/lib/python3.8/site-packages/torch/optim/optimizer.py", line 88, in wrapper
    return func(*args, **kwargs)
  File "/home/vaclavmatejka/anaconda3/envs/dj/lib/python3.8/site-packages/torch/autograd/grad_mode.py", line 28, in decorate_context
    return func(*args, **kwargs)
  File "/home/vaclavmatejka/anaconda3/envs/dj/lib/python3.8/site-packages/torch/optim/sgd.py", line 110, in step
    F.sgd(params_with_grad,
  File "/home/vaclavmatejka/anaconda3/envs/dj/lib/python3.8/site-package

TypeError: object of type 'NoneType' has no len()

In [ ]:
len(list(model.children()))

